In [ ]:
import pathlib
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import PIL

import tensorflow as tf
#from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.optimizers import Adam

In [ ]:
data_dir = "/kaggle/input/handwritten-marathi-character-augmented-dataset/Numbers"
data_dir = pathlib.Path(data_dir)
print(data_dir)

test_data_dir = "/kaggle/input/testingdataset/Numbers"
test_data_dir = pathlib.Path(test_data_dir)
print(test_data_dir)

In [ ]:
ch1 = list(data_dir.glob('C54/*'))
print(ch1[0])
PIL.Image.open(str(ch1[0]))

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

img_height,img_width=260,260
batch_size = 32

# Define data augmentation parameters
train_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.25,
    preprocessing_function=tf.keras.applications.vgg16.preprocess_input
)

val_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.25,    
    preprocessing_function=tf.keras.applications.vgg16.preprocess_input
)

test_datagen = ImageDataGenerator(
    rescale=1./255,
    preprocessing_function=tf.keras.applications.vgg16.preprocess_input
)

# Prepare training dataset with data augmentation
train_ds = train_datagen.flow_from_directory(
    data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training',
    seed=123
)

# Prepare validation dataset without data augmentation
val_ds = val_datagen.flow_from_directory(
    data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation',
    seed=123
)

# Prepare Testing dataset without data augmentation
test_ds = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    seed=123
)

In [ ]:
class_indices = train_ds.class_indices
class_names = {v: k for k, v in class_indices.items()}
print(class_names)

In [ ]:
from tensorflow.keras.applications import EfficientNetB2

NUM_CLASSES=48

model = EfficientNetB2(
    include_top=True,
    weights=None,
    classes=NUM_CLASSES,
    input_shape=(260, 260, 3),
)

In [ ]:
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
model.summary()

In [ ]:
# creating checkpoints
model_path_acc = "/kaggle/working/Numbers_Acc.keras"
model_path_loss = "/kaggle/working/Numbers_Loss.keras"

# Define the checkpoints
checkpoint_acc = ModelCheckpoint(model_path_acc, monitor="val_accuracy", verbose=1, save_best_only=True, mode="max")
checkpoint_loss = ModelCheckpoint(model_path_loss, monitor="val_loss", verbose=1, save_best_only=True, mode="min")

history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=32,
    callbacks=[checkpoint_acc,checkpoint_loss]  
)

In [ ]:
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=3,
    callbacks=[checkpoint_acc,checkpoint_loss]  
)

In [ ]:
from tensorflow.keras.models import load_model
best_model1 = load_model('/kaggle/working/Numbers_Acc.keras')

# best model1
test_loss, test_accuracy = best_model1.evaluate(test_ds)
print(f"Testing Loss: {test_loss}")
print(f"Testing Accuracy: {test_accuracy}")

In [ ]:
from sklearn.metrics import classification_report
import numpy as np

# Predict the labels for the testing dataset
y_pred = best_model1.predict(test_ds)
y_pred_classes = np.argmax(y_pred, axis=1)

# Get the true labels from the validation dataset generator
test_true_labels = test_ds.classes

# Convert one-hot encoded labels to class labels
test_class_labels = list(test_ds.class_indices.keys())

# Generate classification report
classification_rep = classification_report(test_true_labels, y_pred_classes, target_names=test_class_labels)

print("Classification Report:")
print(classification_rep)

In [ ]:
# Confusion Matrix
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix


# Create a confusion matrix
conf_matrix = confusion_matrix(test_true_labels, y_pred_classes)

# Plot the confusion matrix
plt.figure(figsize=(10, 8))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=test_class_labels, yticklabels=test_class_labels)
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Confusion Matrix')
plt.show()

In [ ]:
best_model2 = load_model('/kaggle/working/Numbers_Loss.keras')

# best model2
test_loss, test_accuracy = best_model2.evaluate(test_ds)
print(f"Testing Loss: {test_loss}")
print(f"Testing Accuracy: {test_accuracy}")

In [ ]:
from sklearn.metrics import classification_report
import numpy as np

# Predict the labels for the testing dataset
y_pred = best_model2.predict(test_ds)
y_pred_classes = np.argmax(y_pred, axis=1)

# Get the true labels from the validation dataset generator
test_true_labels = test_ds.classes

# Convert one-hot encoded labels to class labels
test_class_labels = list(test_ds.class_indices.keys())

# Generate classification report
classification_rep = classification_report(test_true_labels, y_pred_classes, target_names=test_class_labels)

print("Classification Report:")
print(classification_rep)

In [ ]:
# Confusion Matrix
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix


# Create a confusion matrix
conf_matrix = confusion_matrix(test_true_labels, y_pred_classes)

# Plot the confusion matrix
plt.figure(figsize=(10, 8))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=test_class_labels, yticklabels=test_class_labels)
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Confusion Matrix')
plt.show()